In [ ]:
!pip install tensorflow==2.5.0 tensorflow-gpu==2.5.0 opencv-python mediapipe sklearn matplotlib

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
#MediaPipe solutions - reconhecimento e desenho dos pontos na mão
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def mediapipe_detection(image, model):
    """ 
    Funcao com objetivo de aplicar a previsao de reconhecimento na imagem desejada 
    Recebe imagem (frame da webcam) e um modelo que sera responsavel pela previsão
    Retorna o frame utilizado e a previsao feita pelo modelo
    A conversao da imagem se faz necessaria para ser tratada pelo modelo (BGR -> RGB)
    A mudanca na propriedade de leitura tem como objetivo salvar memoria
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [ ]:
def draw_landmarks(image, results):
    """
    Funcao que recebe uma imagem (frame) junto com as previsoes feitas pelo modelo e aplica
    sob a imagem o desenho dos pontos necessarios (nao ha motivo para devolver a imagem pois ela
    ja e alterada diretamente).
    """
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [ ]:
#Acessa a webcam - valor (0) representa o hardware
cap = cv2.VideoCapture(0)

#Define o modelo utilizado
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #Seleciona o frame atual da webcam
        ret, frame = cap.read()

        #Processa previsoes
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        #Desenha os pontos
        draw_landmarks(image, results)
        
        #Faz o display do frame
        if ret == True: 
            cv2.imshow('Webcam', image)

            #Encerra a webcam 
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
def extract_keypoints(results):
    '''É preciso guardar todos os pontos lidos pelo modelo em um numpy array
    Este array pose, por exemplo, armazena 33 numpy arrays que guardam, cada um, um ponto
    lido pelo modelo (com 3 coordenadas e visibility). Representando um único frame
    Depois o array é tratado de forma a transformá-lo num único array com todos os pontos.
    É retornado um array que junta todos os pontos lidos pela câmera.'''
    if results.pose_landmarks:      #Verifica se apareceu na câmera uma pessoa (array não vazio)
        pose = []
        for res in results.pose_landmarks.landmark:
            test = np.array([res.x, res.y, res.z, res.visibility])
            pose.append(test)

        pose.flatten()              #Trata os dados de forma a deixá-los todos em um único array de 33 vezes 4 = 132
    else:                           #Caso esteja vazio, cria um numpy array com zeros
        pose = np.zeros(132) 
        
    if results.lefthand_landmarks:  #Verifica se apareceu na câmera a mão esquerda (array não vazio)
        lefthand = []
        for res in results.lefthand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            lefthand.append(test)

        lefthand.flatten()          #Trata os dados de forma a deixá-los todos em um único array de 21 vezes 3 = 63
    else:                           #Caso esteja vazio, cria um numpy array com zeros
        lefthand = np.zeros(63) 

    if results.righthand_landmarks:  #Verifica se apareceu na câmera a mão direita (array não vazio)
        righthand = []
        for res in results.righthand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            righthand.append(test)

        righthand.flatten()         #Trata os dados de forma a deixá-los todos em um único array de 21 vezes 3 = 63
    else:                           #Caso esteja vazio, cria um numpy array com zeros
        righthand = np.zeros(63) 

    if results.face_landmarks:     #Verifica se apareceu na câmera a mão direita (array não vazio)
        face = []
        for res in results.face_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            face.append(test)

        face.flatten()              #Trata os dados de forma a deixá-los todos em um único array de 468 vezes 3 = 1404
    else:                           #Caso esteja vazio, cria um numpy array com zeros
        face = np.zeros(1404) 
    
    return np.concatenate([pose, face, lefthand, righthand]) #Concatenação de todos pontos

In [ ]:
# Caminho que guarda o numpy array com os pontos extraidos
DATA_PATH = os.path.join('MP_Data')

# Sinais que serão detectados
actions = np.array(['ola', 'obrigado', 'teamo'])

# Representa a quantidade de sequências de frames que tem os dados
no_sequences = 30

# Representa a quantidade de frames que cada sequência possui
sequence_lenght = 30

In [ ]:
# Cria pastas para cada sinal definido
# Cada pasta tem 30 pastas representando os videos modelo do sinal
# Cada vídeo contém 30 frames e cada frame 1662 pontos extraidos
for action in actions:
    for sequence in range(no_sequences):
        try: # Cria pastas e subpastas
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except: # Caso já exista passa para próxima pasta
            pass